In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

# Load the CSV file into a DataFrame
file_path = "nov30_fn_run_PF_single_theta_dual_radio_NN.csv"
data = pd.read_csv(file_path)


target_value = "runtime"
target_value = "mse_craft_theta"
# target_value = "mse_single_radio_theta"
# Plot the heatmap
checkpoint_fns = data["checkpoint_fn"].unique() if "checkpoint_fn" in data else [None]
for checkpoint_fn in checkpoint_fns:
    output = f"output/{file_path}/{os.path.basename(os.path.dirname(checkpoint_fn))}/"
    os.makedirs(output, exist_ok=True)
    for segmentation_version in data["segmentation_version"].unique():
        for theta_err in data["theta_err"].unique():
            for frequency in data["frequency"].unique():
                fig, axs = plt.subplots(1, 2, figsize=(12, 6))
                for movement, ax in [("bounce", axs[0]), ("circle", axs[1])]:
                    df = data[
                        (data["movement"] == movement)
                        & (data["theta_err"] == theta_err)
                        & (data["frequency"] == frequency)
                        & (data["segmentation_version"] == segmentation_version)
                    ]
                    if len(df) == 0:
                        continue
                    if checkpoint_fn is not None:
                        df = df[df["checkpoint_fn"] == checkpoint_fn]
                    df = df.copy()

                    # Convert 'N' to integer for sorting
                    df["N"] = df["N"].astype(int)

                    # Sort by 'N' as integers
                    df = df.sort_values(by="N")

                    # Convert 'N' back to string and set as a categorical type with ordered categories
                    df["N"] = df["N"].astype(str)
                    df["N"] = pd.Categorical(
                        df["N"],
                        categories=sorted(df["N"].unique(), key=int),
                        ordered=True,
                    )

                    df["theta_dot_err"] = df["theta_dot_err"].astype(str)

                    # Average other fields over 'mse_craft_theta' with observed=True to avoid the warning
                    heatmap_data = (
                        df.groupby(["N", "theta_dot_err"], observed=True)
                        .agg({target_value: "mean"})
                        .reset_index()
                    )

                    # Pivot the data for the heatmap
                    heatmap_pivot = heatmap_data.pivot(
                        index="theta_dot_err", columns="N", values=target_value
                    )

                    sns.heatmap(
                        heatmap_pivot,
                        annot=True,
                        fmt=".2f",
                        cmap="YlGnBu",
                        cbar_kws={"label": f"Mean {target_value}"},
                        ax=ax,
                    )
                    checkpoint_fn_name = (
                        os.path.basename(os.path.dirname(checkpoint_fn))
                        if checkpoint_fn is not None
                        else "no_NN"
                    )
                    ax.set_title(
                        f"{checkpoint_fn_name} {movement} theta_err:{theta_err}"
                    )
                    ax.set_xlabel("N")
                    ax.set_ylabel("theta_dot_err")
                fig.set_tight_layout(True)
                output_fn = f"{output}/{segmentation_version:0.3f}/{frequency:0.4e}/{movement}_theta_err{theta_err}.png"
                os.makedirs(os.path.dirname(output_fn), exist_ok=True)
                fig.savefig(output_fn)
                plt.close(fig)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "nov27_fn_run_EKF_single_theta_dual_radio.csv"

output = f"output/{file_path}/"
os.makedirs(output, exist_ok=True)
data = pd.read_csv(file_path)

# Plot the heatmap
for p in data["p"].unique():
    for segmentation_version in data["segmentation_version"].unique():
        for frequency in data["frequency"].unique():
            fig, axs = plt.subplots(1, 2, figsize=(12, 6))
            for movement, ax in [("bounce", axs[0]), ("circle", axs[1])]:
                df = data[
                    (data["movement"] == movement)
                    & (data["p"] == p)
                    & (data["dynamic_R"] == 0.0)
                    & (data["frequency"] == frequency)
                    & (data["segmentation_version"] == segmentation_version)
                ].copy()
                if len(df) == 0:
                    continue

                # Convert 'N' to integer for sorting
                df["noise_std"] = df["noise_std"].astype(float)
                df["phi_std"] = df["phi_std"].astype(float)

                # Sort by 'N' as integers
                df = df.sort_values(by="noise_std")
                df = df.sort_values(by="phi_std")

                # Convert 'N' back to string and set as a categorical type with ordered categories
                df["noise_std"] = df["noise_std"].astype(str)
                df["noise_std"] = pd.Categorical(
                    df["noise_std"],
                    categories=sorted(df["noise_std"].unique(), key=float),
                    ordered=True,
                )

                df["phi_std"] = pd.Categorical(
                    df["phi_std"],
                    categories=sorted(df["phi_std"].unique(), key=float),
                    ordered=True,
                )

                # Average other fields over 'mse_craft_theta' with observed=True to avoid the warning
                heatmap_data = (
                    df.groupby(["noise_std", "phi_std"], observed=True)
                    .agg({"mse_craft_theta": "mean"})
                    .reset_index()
                )

                # Pivot the data for the heatmap
                heatmap_pivot = heatmap_data.pivot(
                    index="phi_std", columns="noise_std", values="mse_craft_theta"
                )

                sns.heatmap(
                    heatmap_pivot,
                    annot=True,
                    fmt=".2f",
                    cmap="YlGnBu",
                    cbar_kws={"label": "Mean mse_craft_theta"},
                    ax=ax,
                )
                ax.set_title(f"{movement} p:{p}")
                ax.set_xlabel("noise_std")
                ax.set_ylabel("phi_std")
            fig.set_tight_layout(True)
            fig.savefig(f"{output}/{frequency:0.4e}_{movement}_p{p}.png")
            plt.close(fig)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "nov27_fn_run_PF_single_theta_dual_radio.csv"
data = pd.read_csv(file_path)


output = f"output/{file_path}/"
os.makedirs(output, exist_ok=True)
data = pd.read_csv(file_path)

target_value = "runtime"
target_value = "mse_craft_theta"
# Plot the heatmap
for theta_err in data["theta_err"].unique():
    for frequency in data["frequency"].unique():
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for movement, ax in [("bounce", axs[0]), ("circle", axs[1])]:
            df = data[
                (data["movement"] == movement)
                & (data["theta_err"] == theta_err) * (data["frequency"] == frequency)
            ].copy()
            if len(df) == 0:
                continue

            # Convert 'N' to integer for sorting
            df["N"] = df["N"].astype(int)

            # Sort by 'N' as integers
            df = df.sort_values(by="N")

            # Convert 'N' back to string and set as a categorical type with ordered categories
            df["N"] = df["N"].astype(str)
            df["N"] = pd.Categorical(
                df["N"], categories=sorted(df["N"].unique(), key=int), ordered=True
            )

            df["theta_dot_err"] = df["theta_dot_err"].astype(str)

            # Average other fields over 'mse_craft_theta' with observed=True to avoid the warning
            heatmap_data = (
                df.groupby(["N", "theta_dot_err"], observed=True)
                .agg({target_value: "mean"})
                .reset_index()
            )

            # Pivot the data for the heatmap
            heatmap_pivot = heatmap_data.pivot(
                index="theta_dot_err", columns="N", values=target_value
            )

            sns.heatmap(
                heatmap_pivot,
                annot=True,
                fmt=".2f",
                cmap="YlGnBu",
                cbar_kws={"label": f"Mean {target_value}"},
                ax=ax,
            )
            ax.set_title(f"{movement} theta_err:{theta_err}")
            ax.set_xlabel("N")
            ax.set_ylabel("theta_dot_err")
        fig.set_tight_layout(True)
        fig.savefig(
            f"{output}/{file_path}_{frequency:0.4e}_{movement}_theta_err{theta_err}.png"
        )
        plt.close(fig)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "report_nov6_fn_run_EKF_single_theta_single_radio.csv"
data = pd.read_csv(file_path)

# Plot the heatmap
for p in data["p"].unique():
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    for movement, ax in [("bounce", axs[0]), ("circle", axs[1])]:
        df = data[
            (data["movement"] == movement)
            & (data["p"] == p)
            & (data["dynamic_R"] == 0.0)
        ].copy()

        # Convert 'N' to integer for sorting
        df["noise_std"] = df["noise_std"].astype(float)
        df["phi_std"] = df["phi_std"].astype(float)

        # Sort by 'N' as integers
        df = df.sort_values(by="noise_std")
        df = df.sort_values(by="phi_std")

        # Convert 'N' back to string and set as a categorical type with ordered categories
        df["noise_std"] = df["noise_std"].astype(str)
        df["noise_std"] = pd.Categorical(
            df["noise_std"],
            categories=sorted(df["noise_std"].unique(), key=float),
            ordered=True,
        )

        df["phi_std"] = pd.Categorical(
            df["phi_std"],
            categories=sorted(df["phi_std"].unique(), key=float),
            ordered=True,
        )

        # Average other fields over 'mse_craft_theta' with observed=True to avoid the warning
        heatmap_data = (
            df.groupby(["noise_std", "phi_std"], observed=True)
            .agg({"mse_single_radio_theta": "mean"})
            .reset_index()
        )

        # Pivot the data for the heatmap
        heatmap_pivot = heatmap_data.pivot(
            index="phi_std", columns="noise_std", values="mse_single_radio_theta"
        )

        sns.heatmap(
            heatmap_pivot,
            annot=True,
            fmt=".2f",
            cmap="YlGnBu",
            cbar_kws={"label": "Mean mse_single_radio_theta"},
            ax=ax,
        )
        ax.set_title(f"{movement} p:{p}")
        ax.set_xlabel("noise_std")
        ax.set_ylabel("phi_std")
    fig.set_tight_layout(True)

In [ ]:
sorted([0.1, 0.01, 0.001, 0.0001, 0.2, 0.002, 0.0005, 0.05, 0.02, 0.075, 0.005])

In [ ]:
sorted([0.001, 0.0001, 0.01, 0.1, 0.2, 0.002, 0.0005])